In [1]:
from google.colab import drive
drive.mount('/content/drive')

basic_path = '/content/drive/MyDrive/test/'

Mounted at /content/drive


In [2]:
# 라이브러리 import
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
##
import os
from typing import cast

In [3]:
def get_mean_std(channel, training_dataset):
  if channel.lower() is 'rgb':
    mean_rgb = [np.mean(x.numpy(), axis=(1, 2)) for x,_ in training_dataset]
    std_rgb = [np.std(x.numpy(), axis=(1, 2)) for x,_ in training_dataset]

    mean_r = np.mean([m[0] for m in mean_rgb])
    mean_g = np.mean([m[1] for m in mean_rgb])
    mean_b = np.mean([m[2] for m in mean_rgb])

    std_r = np.mean([s[0] for s in std_rgb])
    std_g = np.mean([s[1] for s in std_rgb])
    std_b = np.mean([s[2] for s in std_rgb])
    return [mean_r, mean_g, mean_b], [std_r, std_g, std_b]
  else:
    mean = [np.mean(x.numpy(), axis=(1, 2)) for x,_ in training_dataset]
    std = [np.std(x.numpy(), axis=(1, 2)) for x,_ in training_dataset]

    return np.mean([m[0] for m in mean]), np.mean([s[0] for s in std])

In [4]:
def get_device():
    return torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
# 모델 구현
MAX_POOLING_LAYER = 'MAX_POOLING'
VGG_types = {
    'VGG16': [64, 64, MAX_POOLING_LAYER, 128, 128, MAX_POOLING_LAYER, 256, 256, 256,
               MAX_POOLING_LAYER, 512, 512, 512, MAX_POOLING_LAYER, 512, 512, 512, MAX_POOLING_LAYER]
}
class MyVGG(nn.Module):
  def __init__(self, size_of_channel, number_of_class, network_type = 'VGG16'):
    super(MyVGG, self).__init__()
    self.size_of_channel = size_of_channel
    self.networks = self._generate_networks_(network_type)
    self.fc_layers = nn.Sequential(
        nn.Linear(512 * 7 * 7, 4096),
        nn.ReLU(),
        nn.Dropout(p=0.5),
        nn.Linear(4096, 4096),
        nn.ReLU(),
        nn.Dropout(p=0.5),
        nn.Linear(4096, number_of_class),
    )
    self._init_weights_()

  def forward(self, x):
    x = self.networks(x)
    x = x.reshape(x.shape[0], -1)
    x = self.fc_layers(x)
    return x

  def _init_weights_(self):
    def __get_all_layers__():
      return self.modules()
    for module in __get_all_layers__():
      if isinstance(module, nn.Conv2d):
        nn.init.kaiming_normal_(module.weight, mode='fan_out', nonlinearity='relu')
        if module.bias is not None:
          nn.init.constant_(module.bias, 0)
      elif isinstance(module, nn.BatchNorm2d):
        nn.init.constant_(module.weight, 1)
        nn.init.constant_(module.bias, 0)
      elif isinstance(module, nn.Linear):
        nn.init.normal_(module.weight, 0, 0.01)
        nn.init.constant_(module.bias, 0)

  def _generate_networks_(self, network_type):
    layers = []
    input_channel_size = self.size_of_channel
    for output_channel_size in VGG_types[network_type]:
      if output_channel_size == MAX_POOLING_LAYER:
        layers += [nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2))]
      else:
        layers += [
            nn.Conv2d(input_channel_size, output_channel_size, kernel_size=(3, 3), padding=(1, 1)),
            nn.BatchNorm2d(output_channel_size),
            nn.ReLU(inplace=True)
        ]
        input_channel_size = output_channel_size
    return nn.Sequential(*layers)


In [6]:
training_dataset = datasets.FashionMNIST(
    root=basic_path + '/data', train=True, download=True, transform=transforms.ToTensor(),
)
test_dataset = datasets.FashionMNIST(
    root=basic_path + '/data', train=False, download=True, transform=transforms.ToTensor(),
)

gray_scale_mean, gray_scale_std = get_mean_std('gray', training_dataset)
gray_scale_transform = transforms.Compose([
  transforms.ToTensor(),
  transforms.Resize(224),
  transforms.Normalize(mean=[gray_scale_mean], std=[gray_scale_std])
])
training_dataset.transform = gray_scale_transform
test_dataset.transform = gray_scale_transform

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [7]:
training_dataloader = DataLoader(training_dataset, batch_size=64)
test_dataloader = DataLoader(test_dataset, batch_size=64)

for X, y in test_dataloader:
  print('Shape of X [N, C, H, W]:', X.shape)
  print('Shape ofy:', y.shape, y.dtype)
  break

Shape of X [N, C, H, W]: torch.Size([64, 1, 224, 224])
Shape ofy: torch.Size([64]) torch.int64


In [8]:
datasets.FashionMNIST.classes

['T-shirt/top',
 'Trouser',
 'Pullover',
 'Dress',
 'Coat',
 'Sandal',
 'Shirt',
 'Sneaker',
 'Bag',
 'Ankle boot']

In [9]:
model_fashion = MyVGG(1, len(datasets.FashionMNIST.classes))
model_fashion

MyVGG(
  (networks): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
   

In [10]:
device = get_device()
print(device)
model_fashion = model_fashion.to(device)

cuda


In [11]:
def train(device, dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    
    for batch, (X, y) in enumerate(dataloader):
      X, y = X.to(device), y.to(device)
      
      pred = model(X.cuda())
      loss = loss_fn(pred, y)
      
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      
      print('.', end='')
      if batch % 100 == 0:
          print()
          loss, current = loss.item(), batch*len(X)
          print(f'loss: {loss:>7f}   [{current:>5d}/{size:>5d}]')

In [12]:
def test(device, dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f'Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loass: {test_loss:>8f}\n')

In [13]:
for epoch in range(5):
  print(f'Epoch {epoch + 1}\n---------------------------------')
  train(device, training_dataloader, model_fashion, nn.CrossEntropyLoss().to(device),
        torch.optim.Adam(model_fashion.parameters(),lr=0.00001))
  test(device, test_dataloader, model_fashion, nn.CrossEntropyLoss().to(device))

Epoch 1
---------------------------------


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


.
loss: 2.464610   [    0/60000]
....................................................................................................
loss: 1.034951   [ 6400/60000]
....................................................................................................
loss: 0.502621   [12800/60000]
....................................................................................................
loss: 0.782980   [19200/60000]
....................................................................................................
loss: 0.687259   [25600/60000]
....................................................................................................
loss: 0.509175   [32000/60000]
....................................................................................................
loss: 0.382671   [38400/60000]
....................................................................................................
loss: 0.634097   [44800/60000]
...........................................

In [14]:
torch.save(model_fashion.state_dict(), 'model_fashion.pth')

In [15]:
from torchsummary import summary

summary(model_fashion, input_size=(1, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]             640
       BatchNorm2d-2         [-1, 64, 224, 224]             128
              ReLU-3         [-1, 64, 224, 224]               0
            Conv2d-4         [-1, 64, 224, 224]          36,928
       BatchNorm2d-5         [-1, 64, 224, 224]             128
              ReLU-6         [-1, 64, 224, 224]               0
         MaxPool2d-7         [-1, 64, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]          73,856
       BatchNorm2d-9        [-1, 128, 112, 112]             256
             ReLU-10        [-1, 128, 112, 112]               0
           Conv2d-11        [-1, 128, 112, 112]         147,584
      BatchNorm2d-12        [-1, 128, 112, 112]             256
             ReLU-13        [-1, 128, 112, 112]               0
        MaxPool2d-14          [-1, 128,